### Step 1: Import Required Libraries

In this step, we import the necessary libraries from PyTorch and Torchvision for building and training our model.


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


### Step 2: Load and Prepare the MNIST Dataset

We download the MNIST handwritten digits dataset, convert the images to tensors, normalize them, and prepare them in batches using DataLoader.


In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

print("Train size:", len(trainset))
print("Test size:", len(testset))


Train size: 60000
Test size: 10000


### Step 3: Define the Convolutional Neural Network (CNN)

Here, we define a simple CNN with two convolutional layers followed by fully connected layers to classify input images into 10 digit classes (0–9).


In [9]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
print(model)


SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [10]:
criterion = nn.CrossEntropyLoss()              # چون خروجی ما 10 کلاس هست (0 تا 9)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # الگوریتم Adam برای بهینه‌سازی


### Step 4: Define Loss Function and Optimizer

We use `CrossEntropyLoss` since this is a multi-class classification problem.  
The Adam optimizer will update the model weights to reduce the loss.

In [11]:
criterion = nn.CrossEntropyLoss()  # For multi-class classification (digits 0–9)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

### Step 5: Train the Model

We train the CNN for 5 epochs on the MNIST training data.  
After each epoch, we print the training loss and accuracy.

In [13]:
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1} - Loss: {running_loss:.3f} - Accuracy: {accuracy:.2f}%")

print("Training finished.")


Epoch 1 - Loss: 146.287 - Accuracy: 95.35%
Epoch 2 - Loss: 43.185 - Accuracy: 98.57%
Epoch 3 - Loss: 28.295 - Accuracy: 99.04%
Epoch 4 - Loss: 21.075 - Accuracy: 99.26%
Epoch 5 - Loss: 15.996 - Accuracy: 99.44%
Training finished.


### Step 6: Evaluate on Test Set

Now we check how well the model performs on unseen data.  
This helps us understand its generalization performance.


In [15]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 98.95%


### Step 7: Save the Trained Model

We save the trained model's weights in a `.pth` file for later use.  
This will allow us to reload it for attacks or unlearning experiments.


In [14]:
torch.save(model.state_dict(), './models/cnn_mnist.pth')


### Step 8: Compute Model Confidence for Each Input

We pass both training and test samples through the model and use softmax to convert the logits into probabilities.  
Then, we extract the maximum confidence value (highest softmax score) for each input image.  
These values help us understand how confident the model is for each prediction.

In [17]:
def get_confidences(model, dataloader):
    confidences = []
    with torch.no_grad():
        for images, _ in dataloader:
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)              # Convert logits to probabilities
            max_conf, _ = torch.max(probs, dim=1)              # Get max probability (confidence)
            confidences.extend(max_conf.cpu().numpy())         # Store confidence values
    return confidences

# Get confidence values for train and test sets
train_conf = get_confidences(model, trainloader)
test_conf = get_confidences(model, testloader)

print(f"Example confidences (train): {train_conf[:5]}")
print(f"Example confidences (test):  {test_conf[:5]}")


Example confidences (train): [0.99999857, 1.0, 0.9991178, 0.9999968, 0.9998772]
Example confidences (test):  [0.99999857, 1.0, 0.99996364, 0.9999999, 0.9999993]


### Step 8: Save Confidence Scores for Later Analysis

We save the extracted confidence values from the training and test sets into `.npy` files.  
This allows us to re-use the data later without having to re-run inference, for example in unlearning or plotting analysis.


In [18]:
import numpy as np
import os

os.makedirs('./results', exist_ok=True)

np.save('./results/train_confidences.npy', np.array(train_conf))
np.save('./results/test_confidences.npy', np.array(test_conf))

print("Confidence logs saved successfully to './results/'")


Confidence logs saved successfully to './results/'
